In [1]:
import json
import re

In [2]:
def build_prompt(q, q_type):
    if q_type == "单选题":
        return f'这是一道{q_type}，请给出你的答案（只需给出单个选项），并进行解析，给出你的理由。\n' + \
                '题目：酸碱灭火器是一种内部装有65％的工业硫酸和碳酸氢钠的水溶液作灭火剂的灭火器，使用时，两种药液混合发生化学反应，产生二氧化碳压力气体，灭火剂在二氧化碳气体压力下喷出进行灭火。下列火灾中，适用酸碱灭火器扑救的是（）。\nA．天然气火灾\nB．金属钠火灾\nC．配电柜火灾\nD．纺织物火灾\n' + \
                '答案：D\n' + \
                '解析：酸碱灭火器是一种内部装有65％的工业硫酸和碳酸氢铀的水溶液作灭火剂的灭火器。使用时，两种药液混合发生化学反应，产生二氧化碳压力气体，灭火剂在二氧化碳气体压力下喷出进行灭火。该类灭火器适用于扑救A类物质的初起火灾，如木、竹、织物、纸张等燃烧的火灾。它不能用于扑救B类物质燃烧的火灾也不能用于扑救C类可燃气体D类轻金属火灾，同时也不能用于带电场合火灾的扑救。\n\n' + \
                f'题目：{q}\n' + \
                '答案：'
    elif q_type == "多选题":
        return f'这是一道{q_type}，请给出你的答案（只需给出选项），并进行解析，给出你的理由。\n' + \
                '题目：关于运行速度的说法，正确的有下列哪些选项？()\nA．运行速度是路面平整、潮湿、自由流状态下，行驶速度累计分布曲线上对应于85%分位值的速度\nB．公路设计应采用运行速度进行检验\nC．相邻路段运行速度之差应小于20km/h\nD．同一路段运行速度与设计速度之差应小于20km/h\n' + \
                '答案：ABC\n' + \
                '解析：A项，运行速度是路面平整、潮湿、自由流状态下，行驶速度累计分布曲线上对应于85%分位值的速度。运行速度考虑了公路上绝大多数驾驶员的交通心理需求，是随着公路路线不断变化的。BCD三项，根据《公路工程技术标准》(JTGB01—2014)第3.5.2条规定，公路设计应采用运行速度进行检验，相邻路段运行速度之差应小于20km/h，同一路段运行速度与设计速度之差宜小于20km/h。《公路项目安全性评价规范》(JTGB05—2015)中有关于运行速度的计算模型。\n\n' + \
                f'题目：{q}\n' + \
                '答案：'
    elif q_type == "判断题":
        return f'这是一道{q_type}，请给出你的答案（只需给出“正确”或“错误”），并进行解析，给出你的理由。\n' + \
                '题目：基本建设投资和更新改造投资的综合范围为总投资50万元以上（含50万元）的项目。\n' + \
                '答案：正确\n' + \
                '解析：基本建设投资指企业、事业、行政单位以扩大生产能力或工程效益为主要目的的新建、扩建工程及有关工作。其综合范围为总投资50万元以上（含50万元）的基本建设项目。更新改造投资指企业、事业单位对原有设施进行固定资产更新和技术改造，以及相应配套的工程和有关工作（不包括大修理和维护工程）。其综合范围为总投资50万元以上（含50万元）的更新改造项目。\n\n' + \
                f'题目：{q}\n' + \
                '答案：'
    raise Exception(f"{q_type} error")

In [3]:
tiku_files = { "safety_roduction": [
                "cleaned_2023年中级注册安全工程师《安全生产技术基础》题库",
                "cleaned_建筑施工项目负责人B证模拟练习题",
                "cleaned_陆上石油天然气开采安全管理人员复习题",
                "cleaned_煤矿安全生产和管理能力",
                "cleaned_天然气安全生产管理人员",
                "cleaned_铁路安全试题"
            ],
        
        "building_engineering": [
                "cleaned_2020年12月全国二级建造师《水利水电工程管理与实务》真题精选及详解",
                "cleaned_2022二建《管理》真题",
                "cleaned_2023年二级造价工程师《造价管理》模拟试卷3",
                "cleaned_2024年注册土木工程师（道路工程）《专业考试》题库【历年真题＋章节题库＋模拟试题】(1)",
                "cleaned_2024年注册土木工程师（岩土）《专业基础考试》题库【历年真题（部分视频讲解）＋章节题库】(1)",
                "cleaned_2024年咨询工程师（投资）《宏观经济政策与发展规划》模拟试题及详解（二）",
                "cleaned_消防救援题库",
                "cleaned_消防设施操作员（初中高级）题库",
                "cleaned_消防职业技能鉴定理论"
            ],
        
        "economics_financial": [
                "cleaned_2022年中级经济师《工商》考前押题卷",
                "cleaned_2022年中级经济师《人力》题库",
                "cleaned_2023反假币",
                "cleaned_2024年经济师《经济基础知识（中级）》考前模拟题及详解（二）",
                "cleaned_2024年银行业专业人员职业资格考试《银行业法律法规与综合能力（初级）》机考题库【真题精选＋章节题库】",
                "cleaned_保险",
                "cleaned_金融学题库",
                "cleaned_银行从业法律法规与综合能力题库",
                "cleaned_银行业法律法规与综合能力",
                "cleaned_中级经济师金融知识"
            ]
       }

In [ ]:
file_path = "../../data/QualBench.json"

with open(file_path, "r", encoding="utf8") as f:
    datas = json.load(f)


In [7]:
datas[128]

{'idx': 128,
 'question_type': '单选题',
 'domain': '安全生产',
 'question': '根据《特种设备安全监察条例》，大型游乐设施是指用于经营目的，承载乘客游乐的设施，其范围规定为运行高度距地面高于或者等于2m，或者设计最大运行线速度大于或等于（）的载人大型游乐设施。\nA．1m/s\nB．3m/s\nC．4m/s\nD．2m/s',
 'answer': 'D',
 'explanation': '大型游乐设施，是指用于经营目的，承载乘客游乐的设施，其范围规定为设计最大运行线速度大于或者等于2m/s，或者运行高度距地面高于或者等于2m的载人大型游乐设施。用于体育运动、文艺演出和非经营活动的大型游乐设施除外。'}

In [8]:
# models = ["chatglm3-6b-chat",
#          "baichuan2-7b-chat",
#          "qwen2.5-7b-instruct",
#          "mistral-7b-instruct-v0.3",
#          "deepseek-v2-lite-chat"]
# models = ["FinLLM_lora_computing",
#           "FinLLM_lora_consulting",
#           "FinLLM_lora_retrieval", "FinLLM_lora_task"]

# models = ["Mistral_prompt_demo"]
# # models = ["qwen14b_shuffled"]
# # models = ["qwen7b_prompt", "qwen14b_prompt_demo"]

models = ["llama14b"]
opts = "ABCDEDFHIJKLMN"
res = {}
error = 0
for m in models:
    with open(f"res_{m}.jsonl", "r", encoding="utf8") as f:
        for line in f:
            d = json.loads(line)
            try:
                resp = d["resp"].split("题目：")[2].split("答案：")[1]
                tmp = resp.split("解析：")
            except:
                continue
            ans = ""

            if d["question_type"] == "判断题":
                if "正确" in tmp[0]:
                    ans = "正确"
                else:
                    ans = "错误"
            else:
                for s in tmp[0].strip().upper():
                    if s in opts:
                        ans += s
            if ans == "":
                # ans = extract_answer_with_gpt4o(resp)
                # print(ans)
                # print(d)
                error += 1
            try:
                datas[d["idx"]][m] = {}
                datas[d["idx"]][m]["answer"] = ans
                datas[d["idx"]][m]["explan"] = tmp[1].strip() if len(tmp) > 1 else ""
            except Exception as e:
                # print(e)
                pass
            tmp = []

            # if error > 5:
            #     break
    print(error)


0


In [9]:
datas[0]

{'idx': 0,
 'question_type': '单选题',
 'domain': '安全生产',
 'question': '机械设备运动部分是最危险的部位，尤其是那些操作人员易接触的零部件，下列针对不同机械设备转动部位的危险所采取的安全防护措施中，正确的是（）。\nA．针对轧钢机，在验式机轴处采用锥形防护罩防护\nB．针对辊式输送机，在驱动轴上游安装防护罩防护\nC．针对啮合齿轮，齿轮转动机构采用半封闭防护\nD．针对手持式砂轮机，在磨削区采用局部防护',
 'answer': 'A',
 'explanation': 'B项，辊式输送机应该在驱动轴的下游安装防护罩。',
 'llama14b': {'answer': 'C',
  'explan': '机械设备运动部分是最危险的部位，尤其是那些操作人员易接触的零部件。针对不同机械设备转动部位的危险所采取的安全防护措��'}}

In [10]:
with open(f"res_{m}.jsonl", "r", encoding="utf8") as f:
    for line in f:
        d = json.loads(line)
        print(d)
        break

{'idx': 0, 'question_type': '单选题', 'domain': '安全生产', 'question': '机械设备运动部分是最危险的部位，尤其是那些操作人员易接触的零部件，下列针对不同机械设备转动部位的危险所采取的安全防护措施中，正确的是（）。\nA．针对轧钢机，在验式机轴处采用锥形防护罩防护\nB．针对辊式输送机，在驱动轴上游安装防护罩防护\nC．针对啮合齿轮，齿轮转动机构采用半封闭防护\nD．针对手持式砂轮机，在磨削区采用局部防护', 'answer': 'A', 'explanation': 'B项，辊式输送机应该在驱动轴的下游安装防护罩。', 'resp': '这是一道单选题，请给出你的答案（只需给出单个选项），并进行解析，给出你的理由。\n题目：酸碱灭火器是一种内部装有65％的工业硫酸和碳酸氢钠的水溶液作灭火剂的灭火器，使用时，两种药液混合发生化学反应，产生二氧化碳压力气体，灭火剂在二氧化碳气体压力下喷出进行灭火。下列火灾中，适用酸碱灭火器扑救的是（）。\nA．天然气火灾\nB．金属钠火灾\nC．配电柜火灾\nD．纺织物火灾\n答案：D\n解析：酸碱灭火器是一种内部装有65％的工业硫酸和碳酸氢铀的水溶液作灭火剂的灭火器。使用时，两种药液混合发生化学反应，产生二氧化碳压力气体，灭火剂在二氧化碳气体压力下喷出进行灭火。该类灭火器适用于扑救A类物质的初起火灾，如木、竹、织物、纸张等燃烧的火灾。它不能用于扑救B类物质燃烧的火灾也不能用于扑救C类可燃气体D类轻金属火灾，同时也不能用于带电场合火灾的扑救。\n\n题目：机械设备运动部分是最危险的部位，尤其是那些操作人员易接触的零部件，下列针对不同机械设备转动部位的危险所采取的安全防护措施中，正确的是（）。\nA．针对轧钢机，在验式机轴处采用锥形防护罩防护\nB．针对辊式输送机，在驱动轴上游安装防护罩防护\nC．针对啮合齿轮，齿轮转动机构采用半封闭防护\nD．针对手持式砂轮机，在磨削区采用局部防护\n答案：C\n解析：机械设备运动部分是最危险的部位，尤其是那些操作人员易接触的零部件。针对不同机械设备转动部位的危险所采取的安全防护措��'}


In [11]:
datas[50]

{'idx': 50,
 'question_type': '单选题',
 'domain': '安全生产',
 'question': '人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是（）。\nA．以机为主\nB．以人为主\nC．人机同等\nD．人机共体',
 'answer': 'A',
 'explanation': '在自动化系统中，则以机为主体，机器的正常运转完全依赖于闭环系统的机器自身的控制，人只是一个监视者和管理者，监视自动化机器的工作。只有在自动控制系统出现差错时，人才进行干预，采取相应的措施。该系统的安全性主要取决于机器的本质安全性、机器的冗余系统是否失灵以及人处于低负荷时的应急反应变差等情形。',
 'llama14b': {'answer': 'D',
  'explan': '人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是人机共体，即人和机器在一个整体中，相��'}}

In [12]:
filtered_data = datas
cleaned = []
already = set()
for d in filtered_data:
    if d['question'] in already:
        continue
    # Use the new keys according to the new structure
  #  d['input'] = build_prompt(d["question"], d["question_type"])
    if d["question_type"] == "判断题" and d["answer"] in {"正确", "错误"}:
        cleaned.append(d)
        already.add(d['question'])
    if d["question_type"] in {"单选题", "多选题"}:
        right = True
        for s in d["answer"]:
            if not s in opts:
                right = False
                break
        if right:
            cleaned.append(d)
            already.add(d['question'])
print(len(cleaned))

17298


In [13]:
file_path = "./shuffled_dataset.jsonl"

shuffled_data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line_number, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            print(f"Skipping empty line at line {line_number}")
            continue
        try:
            shuffled_data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON at line {line_number}: {str(e)}")
            continue


In [14]:
shuffled_data[90]

{'question': '造成机房电气火灾的主要因素有超负荷、静电、雷击、人为操作失误等，遇到机房电气火灾，应优先选用（）。\nA．酸碱灭火器\nB．水基灭火器\nC．二氧化碳灭火器\nD．泡沫灭火器',
 'answer': 'C',
 'explan': 'A项，不能用水扑灭的火灾主要包括：电气火灾未切断电源前不能用水扑救，因为水是良导体，容易造成触电。B项，泡沫灭火器适合扑救脂类、石油产品等B类火灾以及木材等A类物质的初起火灾，但不能扑救B类水溶性火灾，也不能扑救带电设备及C类和D类火灾。C项，二氧化碳灭火器，更适宜于扑救600V以下带电电器、贵重设备、图书档案、精密仪器仪表的初起火灾，以及一般可燃液体的火灾。D项，酸碱灭火器不能用于扑救B类物质燃烧的火灾，也不能用于扑救C类可燃气体或D类轻金属火灾，同时也不能用于带电场合火灾的扑救。',
 'type': '单选题',
 'idx': 94,
 'domain': 'safety_roduction',
 'sub_domain': '安全生产'}

In [15]:
# # Filter data to only include entries whose 'idx' exists in shuffled_data
# shuffled_idxs = set(item['idx'] for item in shuffled_data)
# filtered_data = [item for item in datas if item.get('idx') in shuffled_idxs]
# print(f"Number of filtered items: {len(filtered_data)}")

In [16]:
filtered_data = datas

In [17]:
filtered_data[50]

{'idx': 50,
 'question_type': '单选题',
 'domain': '安全生产',
 'question': '人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是（）。\nA．以机为主\nB．以人为主\nC．人机同等\nD．人机共体',
 'answer': 'A',
 'explanation': '在自动化系统中，则以机为主体，机器的正常运转完全依赖于闭环系统的机器自身的控制，人只是一个监视者和管理者，监视自动化机器的工作。只有在自动控制系统出现差错时，人才进行干预，采取相应的措施。该系统的安全性主要取决于机器的本质安全性、机器的冗余系统是否失灵以及人处于低负荷时的应急反应变差等情形。',
 'llama14b': {'answer': 'D',
  'explan': '人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是人机共体，即人和机器在一个整体中，相��'}}

In [18]:
# filtered_data = [item for item in datas if "qwen14b_shuffled" in item]
# print(f"Number of items with 'qwen14b_shuffled': {len(filtered_data)}")

In [19]:
# # Filter parsed_data to only keep items with domain type '金融'
# filtered_data = [item for item in datas if item.get('domain') == '经济金融']
# print(f"Number of items with domain type '金融': {len(filtered_data)}")

In [20]:
cleaned = []
already = set()
for d in filtered_data:
    if d['question'] in already:
        continue
    # Use the new keys according to the new structure
    d['input'] = build_prompt(d["question"], d["question_type"])
    if d["question_type"] == "判断题" and d["answer"] in {"正确", "错误"}:
        cleaned.append(d)
        already.add(d['question'])
    if d["question_type"] in {"单选题", "多选题"}:
        right = True
        for s in d["answer"]:
            if not s in opts:
                right = False
                break
        if right:
            cleaned.append(d)
            already.add(d['question'])
print(len(cleaned))

17298


In [21]:
cleaned[50]

{'idx': 50,
 'question_type': '单选题',
 'domain': '安全生产',
 'question': '人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是（）。\nA．以机为主\nB．以人为主\nC．人机同等\nD．人机共体',
 'answer': 'A',
 'explanation': '在自动化系统中，则以机为主体，机器的正常运转完全依赖于闭环系统的机器自身的控制，人只是一个监视者和管理者，监视自动化机器的工作。只有在自动控制系统出现差错时，人才进行干预，采取相应的措施。该系统的安全性主要取决于机器的本质安全性、机器的冗余系统是否失灵以及人处于低负荷时的应急反应变差等情形。',
 'llama14b': {'answer': 'D',
  'explan': '人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是人机共体，即人和机器在一个整体中，相��'},
 'input': '这是一道单选题，请给出你的答案（只需给出单个选项），并进行解析，给出你的理由。\n题目：酸碱灭火器是一种内部装有65％的工业硫酸和碳酸氢钠的水溶液作灭火剂的灭火器，使用时，两种药液混合发生化学反应，产生二氧化碳压力气体，灭火剂在二氧化碳气体压力下喷出进行灭火。下列火灾中，适用酸碱灭火器扑救的是（）。\nA．天然气火灾\nB．金属钠火灾\nC．配电柜火灾\nD．纺织物火灾\n答案：D\n解析：酸碱灭火器是一种内部装有65％的工业硫酸和碳酸氢铀的水溶液作灭火剂的灭火器。使用时，两种药液混合发生化学反应，产生二氧化碳压力气体，灭火剂在二氧化碳气体压力下喷出进行灭火。该类灭火器适用于扑救A类物质的初起火灾，如木、竹、织物、纸张等燃烧的火灾。它不能用于扑救B类物质燃烧的火灾也不能用于扑救C类可燃气体D类轻金属火灾，同时也不能用于带电场合火灾的扑救。\n\n题目：人机系统是由相互作用、相互依存的人和机器两个子系统构成，能完成特定目标的一个整体系统。在自动化系统中，人机功能分配的原则是（）。\nA．以机为主\nB．以人为主\nC．人机同等\nD．人机共体\n答案：'}

In [22]:
# true_label = [entry["answer"] for entry in cleaned]
# predicted_label = [entry["FinLLM"]["answer"] for entry in cleaned]

In [23]:
# for m in models:
#     predicted_label = [entry[m]["answer"] for entry in cleaned]
#     # You can now use predicted_label for further analysis per model

In [24]:
scores = {m:0 for m in models}
for r in cleaned:
    try:
        for m in models:
            if r[m]["answer"] == r["answer"]:
                scores[m] += 1
    except:
        pass

print(scores)
for m, s in scores.items():
    print(f"{m} acc:\t{scores[m]/len(datas)*100:.2f}%")

{'llama14b': 6513}
llama14b acc:	37.65%


In [25]:
file_path = "./shuffled_dataset.jsonl"

shuffled_data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line_number, line in enumerate(f, 1):
        line = line.strip()
        if not line:
            print(f"Skipping empty line at line {line_number}")
            continue
        try:
            shuffled_data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON at line {line_number}: {str(e)}")
            continue


In [26]:
def custom_accuracy_score(y_true, y_pred):
    """
    Custom accuracy function that considers predictions correct if they contain the same characters,
    regardless of their order.

    Args:
        y_true (list): List of true labels.
        y_pred (list): List of predicted labels.

    Returns:
        float: Custom accuracy score.
    """
    correct = sum(sorted(true) == sorted(pred) for true, pred in zip(y_true, y_pred))
    return correct / len(y_true)

for m in models:
    y_true = [entry["answer"] for entry in cleaned]
    y_pred = [entry[m]["answer"] for entry in cleaned]
    acc = custom_accuracy_score(y_true, y_pred)
    print(f"{m} acc:\t{acc * 100:.2f}%")

KeyError: 'llama14b'

In [ ]:
scores

{'baichuan13b': 7910}

In [ ]:
for t, _ in tiku_files.items():
    print(f"----domain: {t}-----")
    scores = {m:0 for m in models}
    total = 0
    for r in cleaned:
        if r['domain'] == t:
            total += 1
            for m in models:
                if r[m]["answer"] == r["answer"]:
                    scores[m] += 1
    for m, s in scores.items():
        print(f"{m} acc:\t{scores[m]/total*100:.2f}%")#right: {scores[m]} total: {total}")

----domain: safety_roduction-----


ZeroDivisionError: division by zero